# KG curation, interlinking and multilinguality

In this notebook we look at how embeddings can be used in curation of Knowledge Graphs, in particular in tasks such as graph completion and alignment.

## KG completion

Knowledge Graph completion is the task of predicting whether an existing, incomplete, graph should add a vertix between two specific nodes. For example, in DBpedia, you may want to generate new links between pages and categories.

Although embeddings for KGs are more suitable for this kind of task, word (and cross-modal) embeddings can also provide valuable input.

## Multilingual KG alignment

If you have multiple KGs that need to be aligned, you may be able to use *embedding alignment techniques*.

## Linear alignment
The most straightforward alignment between two embedding spaces can be achieved by using a *translation matrix*.

To calculate the translation matrix, you need a **dictionary** that provides mappings for a subset of your vocabularies.

You can then use existing linear algorithms to calculate the pseudo inverse.

For best results, it is recommended to use parallel corpora (so that the same words are encoded in similar ways) or very large corpora.

In the following example, since we do not have a Spanish version of the kcap corpus, we use pre-generated embeddings for the most frequent 5K lemmas in the *United Nations parallel corpus*. We first load the vectors. 

In [0]:
%cd /content
!git clone https://github.com/HybridNLP2018/tutorial.git
from tutorial.scripts.swivel import vecs
import os
import pandas as pd
import numpy as np
from IPython.display import display

In [0]:
en_path = '/content/tutorial/datasamples/UNv1.0/en_lemma_5k/'
es_path = '/content/tutorial/datasamples/UNv1.0/es_lemma_5k/'
en_vecs = vecs.Vecs(en_path + 'vocab.txt', 
            en_path + 'vecs.bin')
es_vecs = vecs.Vecs(es_path + 'vocab.txt',
            es_path + 'vecs.bin')

Let's check a couple of words in each embedding space as we have done in previous notebooks:

In [0]:
import pandas as pd
pd.DataFrame(en_vecs.k_neighbors('knowledge'))

In [0]:
pd.DataFrame(es_vecs.k_neighbors('conocimiento'))

Besides the embeddings for English and Spanish, we also provide a **dictionary** that was generated automatically to map 1K English lemmas into Spanish.

In [0]:
%ls /content/tutorial/datasamples/UNv1.0/
en2es_dict_path = '/content/tutorial/datasamples/UNv1.0/en2es-lemma-dict-1k.txt'
!head -n 5 {en2es_dict_path}

Let's load the dictionary into a python object.

In [0]:
def load_dict(path, invert=False):
    result = {}
    with open(path, 'r') as lines:
        for line in lines:
            (key, val) = line.split(':')
            if invert:
                result[val.strip('\n')] = key
            else: 
                result[key] = val.strip('\n')
    return result

In [0]:
en2es = load_dict(en2es_dict_path)
es2en = load_dict(en2es_dict_path, invert=True)
len(en2es), len(es2en)

We can see from the reported numbers that some English lemmas were mapped to the same Spanish lemma.

Let's inspect some of the entries in the dictionary:

In [0]:
min = 5
max = min + 5
for en in list(en2es)[min:max]:
    print(en, '->', en2es[en])
print('')
for es in list(es2en)[min:max]:
    print(es, '->', es2en[es])

In order to create the translation matrix, we need to create two **aligned** matrices:
  - $M_{en}$ will contain $n$ English embeddings from the dictionary
  - $M_{es}$ will contain $n$ Spanish embeddings from the dictionary
  
However, since the dictionary was generated automatically, it may be the case that some of the entries in the dictionary are not in the English or Spanish vocabularies. We only need the `id`s in the respective `vecs`:

In [0]:
en_dict_ids = []
es_dict_ids = []
es_dict_voc = []
for es in es2en:
    es_id = es_vecs.word_to_idx.get(es)
    en_id = en_vecs.word_to_idx.get(es2en[es])
    if en_id and es_id :
        es_dict_voc.append(es)
        en_dict_ids.append(en_id)
        es_dict_ids.append(es_id)
print(len(en_dict_ids), len(es_dict_ids))

From the 1K dictionary entries, only $477$ pairs were both in the English and the Spanish `vecs`. In order to verify that the translation works, we can split this into $450$ pairs that we will use to calculate the translation matrix and we keep the remaining $27$ for testing:

In [0]:
train_en_dict_ids = en_dict_ids[:450]
train_es_dict_ids = es_dict_ids[:450]
test_en_ids = en_dict_ids[450:] 
test_es_ids = es_dict_ids[450:]
print(len(train_en_dict_ids), len(test_en_ids))

Before calculating the translation matrix, let's verify that we need one. We chose 3 example words:
  - *conocimiento*  and *proporcionar* are in the in the training set, 
  - *tema* is in the test set
  
For each word, we get:
 - the $5$ Spanish neighbors for the English vector
 - the $5$ Spanish neighbors for the Spanish translation according to the dictionary

In [0]:
es_examples = ['conocimientos', 'proporcionar', 'tema']
from IPython.display import display
for i, es in enumerate(es_examples):
    print(es, '->', es2en[es])
    print('top k for Spanish vector in English vector space:')
    k = 5
    df1 = pd.DataFrame(en_vecs.k_neighbors(es_vecs.lookup(es), k=k, result_key_suffix='_es_vec'))
    print('top k for English translation in English vector space:')
    df2 = pd.DataFrame(en_vecs.k_neighbors(es2en[es], k=k, result_key_suffix='_en'))
    df3 = pd.concat([df1, df2], axis=1)
    #print(df3)
    display(df3)
    print('')

Clearly, simply using the Spanish vector in the English space does not work. Let's get the matrices:

In [0]:
m_en = en_vecs.vecs[train_en_dict_ids]
m_es = es_vecs.vecs[train_es_dict_ids]
print(m_en.shape, m_es.shape)

As expected, we get two matrices of $450 \times 300$, since embeddings are of dimension $300$ and we have $450$ training examples. Now, we can calculate the translation matrix and define a method for linearly translating a point in the Spanish embedding space into a point in the English embedding space.

In [0]:
tm_es2en = np.linalg.pinv(m_es).dot(m_en)
def es_vec_to_en_vec(es_vec):
    return np.dot(es_vec, tm_es2en)
print(tm_es2en.shape)

As we can see, the translation matrix is just a $300 \times 300$ matrix.

Now that we have the translation matrix, let's inspect the example words to see how it performs:

In [0]:
for i, es in enumerate(es_examples):
    print(es, '->', es2en[es])
    k = 5
    print('\t%s: Spanish vector for "%s" in English vector space' % ('es_vec', es))
    df1 = pd.DataFrame(en_vecs.k_neighbors(es_vecs.lookup(es), k=k, result_key_suffix='_es_vec'))
    print('\t%s: English vector for "%s" in English vector space' % ('en', es2en[es]))
    df2 = pd.DataFrame(en_vecs.k_neighbors(es2en[es], k=k, result_key_suffix='_en'))
    print('\t%s: Spanish vector for "%s" *mapped* to English vector space using tm_es2en' % ('tm_es_vec', es))
    df3 = pd.DataFrame(en_vecs.k_neighbors(es_vec_to_en_vec(es_vecs.lookup(es)), k=k, result_key_suffix='_tm_es_vec'))
    df4 = pd.concat([df1,df2,df3], axis=1)
    display(df4)
    print('')

## Non-linear alignment

The linear alignment seems to work OK for this set of embeddings. In our experience, when dealing with larger vocabularies (and vocabularies mixing lemmas and concepts), this approach does not scale, since the number of parameters is limited to the $d \times d$ translation matrix.

For such cases it is possible to follow the same approach, but instead of deriving a pseudo-inverse matrix, we train a neural network to learn a non-linear translation function. The non-linearities can be introduced by using activation functions such as ReLUs.

See  [Towards a Vecsigrafo: Portable Semantics in Knowledge-based Text Analytics](https://pdfs.semanticscholar.org/b0d6/197940d8f1a5fa0d7474bd9a94bd9e44a0ee.pdf) for more details.

## Example: Cross-modal embeddings

In [Thoma, S., Rettinger, A., & Both, F. (2017). Towards Holistic Concept Representations: Embedding Relational Knowledge, Visual Attributes, and Distributional Word Semantics. In International Semantic Web Conference. Vienna, Austria.](https://pdfs.semanticscholar.org/413e/b0b519ac18ec86aaa290c86553291fae7ea2.pdf), a cross-modal embedding is generated for a 1538 concepts.  

![Cross-modal embeddings](https://github.com/HybridNLP2018/tutorial/blob/master/images/cross-modal-embedding.PNG?raw=1)

As part of their evaluations, the authors studied the problem of entity-type prediction (a subtask of KG completion), using a subgraph of DBpedia that provided coverage for the 1538 concepts. Their results were:

![TriM1538 entity-type prediction results](https://github.com/HybridNLP2018/tutorial/blob/master/images/TriM1538-entity-type-pred-results.PNG?raw=1)

The results show a clear improvement when using multi-modal embeddings, compared to just using the KG embeddings.

In [notebook 08](https://colab.research.google.com/github/HybridNLP2018/tutorial/blob/master/08_scientific_information_management.ipynb) of this tutorial you will see another possible way of exploiting cross-modality.